In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
from os.path import join, isdir

import pandas as pd
import geopandas as gpd
import numpy as np

from covidcaremap.data import (processed_data_path, 
                               external_data_path,
                               published_data_path,
                               local_data_path)

from rapidfuzz import fuzz, process

In [3]:
ccm_matching_df = pd.read_csv(external_data_path('covidcaremap_facility_matching_20200407.csv'))

In [4]:
ccm_matching_df.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID,DH name lookup,DH address lookup,HCRIS name lookup,HCRIS address lookup
0,IU HEALTH UNIVERSITY HOSPITAL,GENERAL ACUTE CARE,550 UNIVERSITY BLVD,NaN,INDIANAPOLIS,IN,46202,MARION,39.775283,-86.176562,100,NaN,NaN,100,NaN,NaN,NaN,NaN
1,HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVERLY,REHABILITATION,"800 CUMMINGS CENTER, 1ST FL, UNIT147",NaN,BEVERLY,MA,1915,ESSEX,42.560987,-70.887236,10001915,NaN,NaN,10001915,NaN,NaN,NaN,NaN
2,ST. JOSEPH'S HOSPITAL,CRITICAL ACCESS,12886 TROXLER AVENUE,NaN,HIGHLAND,IL,62249,MADISON,38.754584,-89.667598,100062249,NaN,NaN,100062249,NaN,NaN,NaN,NaN
3,HOSP SAN CARLOS BORROMEO,GENERAL ACUTE CARE,550 CONCEPCION VERA AYALA,NaN,MOCA,PR,676,MOCA,18.390194,-67.109969,1000676,4294.0,400111.0,1000676,Hospital San Carlos Borromeo,"Calle Concepcion Vera Ayala #550 S, Moca, PR",HOSPITAL SAN CARLOS BORROMEO,"68, MOCA, PR"
4,RIVERVIEW MEDICAL CENTER,GENERAL ACUTE CARE,1 RIVERVIEW PLAZA,NaN,RED BANK,NJ,7701,MONMOUTH,40.353982,-74.063843,10007701,2512.0,310034.0,10007701,Riverview Medical Center,"1 Riverview Plz, Red Bank, NJ",RIVERVIEW MEDICAL CENTER,"RIVERVIEW PLAZA, RED BANK, NJ"


In [5]:
ccm_csv_df = pd.read_csv(
    published_data_path('us_healthcare_capacity-facility-CovidCareMap.csv')
)

In [6]:
ccm_csv_df.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,...,ICU Bed Occupancy Rate,Staffed All Beds - SOURCE,Staffed ICU Beds - SOURCE,Licensed All Beds - SOURCE,All Bed Occupancy Rate - SOURCE,ICU Bed Occupancy Rate - SOURCE,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID
0,IU HEALTH UNIVERSITY HOSPITAL,GENERAL ACUTE CARE,550 UNIVERSITY BLVD,NaN,INDIANAPOLIS,IN,46202,MARION,39.775283,-86.176562,...,NaN,None,None,NaN,None,None,100,NaN,NaN,100.0
1,HEALTHSOUTH NEW ENGLAND REHABILIATION AT BEVERLY,REHABILITATION,"800 CUMMINGS CENTER, 1ST FL, UNIT147",NaN,BEVERLY,MA,1915,ESSEX,42.560987,-70.887236,...,NaN,None,None,HIFLD-BEDS,None,None,10001915,NaN,NaN,10001915.0
2,ST. JOSEPH'S HOSPITAL,CRITICAL ACCESS,12886 TROXLER AVENUE,NaN,HIGHLAND,IL,62249,MADISON,38.754584,-89.667598,...,NaN,None,None,NaN,None,None,100062249,NaN,NaN,100062249.0
3,HOSP SAN CARLOS BORROMEO,GENERAL ACUTE CARE,550 CONCEPCION VERA AYALA,NaN,MOCA,PR,676,MOCA,18.390194,-67.109969,...,0.73,HCRIS-Total Staffed Beds,HCRIS-ICU Total Staffed Beds,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,1000676,4294.0,400111.0,1000676.0
4,RIVERVIEW MEDICAL CENTER,GENERAL ACUTE CARE,1 RIVERVIEW PLAZA,NaN,RED BANK,NJ,7701,MONMOUTH,40.353982,-74.063843,...,0.42,HCRIS-Total Staffed Beds,HCRIS-ICU Total Staffed Beds,DH-NUM_LICENS,HCRIS-Total Bed Occupancy Rate,HCRIS-ICU Occupancy Rate,10007701,2512.0,310034.0,10007701.0


In [7]:
fl_hosp = pd.read_csv(external_data_path('FL_Hospital_BedsHospital1_crosstab_0503.csv'), encoding='utf-8')
fl_icu = pd.read_csv(external_data_path('FL_ICU_BedsHospital1_crosstab_0503.csv'), encoding='utf-8')

In [8]:
fl_hosp.drop(0,inplace=True), fl_icu.drop(0,inplace=True)

(None, None)

In [9]:
fl_hosp['Total Staffed Bed Capacity'] = fl_hosp['Total Staffed Bed Capacity'].apply(lambda x: int(x.replace(',','')))
fl_icu['Total AdultICU Capacity'] = fl_icu['Total AdultICU Capacity'].apply(lambda x: int(x.replace(',','')))
fl_hosp['Bed Census'] = fl_hosp['Bed Census'].apply(lambda x: int(x.replace(',','')))
fl_icu['Adult ICU Census'] = fl_icu['Adult ICU Census'].apply(lambda x: int(x.replace(',','')))

In [10]:
fl_hosp.head()

,County,FileNumber,ProviderName,Bed Census,Available,Available Capacity,Total Staffed Bed Capacity
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,673,373,35.66%,1046
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,312,123,28.28%,435
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,45,5,10.00%,50
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,41,40,49.38%,81
5,ALACHUA,23960071,SELECT SPECIALTY HOSPITAL GAINESVILLE,42,6,12.50%,48


In [11]:
fl_icu.head()

,County,FileNumber,ProviderName,Adult ICU Census,Available Adult ICU,Available Adult ICU%,Total AdultICU Capacity,Pediatric ICU Census,Available Pediatric ICU,Available Pediatric ICU%,Total PediatricICU Capacity
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,190,70,26.92%,260,30,17,36.17%,47
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,19,29,60.42%,48,0,0,NaN,0
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,0,0,NaN,0,0,0,NaN,0
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,0,0,NaN,0,0,0,NaN,0
5,ALACHUA,23960071,SELECT SPECIALTY HOSPITAL GAINESVILLE,0,0,NaN,0,0,0,NaN,0


In [13]:
ccm_matching_df.columns

Index(['Name', 'Hospital Type', 'Address', 'Address_2', 'City', 'State',
       'Zipcode', 'County', 'Latitude', 'Longitude', 'CCM_ID', 'DH-ID',
       'HCRIS-ID', 'HIFLD-ID', 'DH name lookup', 'DH address lookup',
       'HCRIS name lookup', 'HCRIS address lookup'],
      dtype='object')

In [15]:
ccm_fl = ccm_matching_df[ccm_matching_df['State'] == 'FL']
ccm_fl.head()

,Name,Hospital Type,Address,Address_2,City,State,Zipcode,County,Latitude,Longitude,CCM_ID,DH-ID,HCRIS-ID,HIFLD-ID,DH name lookup,DH address lookup,HCRIS name lookup,HCRIS address lookup
194,FLORIDA HOSPITAL ALTAMONTE,GENERAL ACUTE CARE,601 E ALTAMONTE DR,NaN,ALTAMONTE SPRINGS,FL,32701,SEMINOLE,28.665379,-81.370158,10532701,NaN,NaN,10532701,NaN,NaN,NaN,NaN
283,FISHERMEN'S HOSPITAL,GENERAL ACUTE CARE,3301 OVERSEAS HWY,NaN,MARATHON,FL,33050,MONROE,24.710362,-81.094436,9733050,797.0,101312.0,9733050,Fishermens Community Hospital,"3301 Overseas Hwy, Marathon, FL",FISHERMENS HOSPITAL,"3301 OVERSEAS HIGHWAY, MARATHON, FL"
310,FLORIDA HOSPITAL APOPKA,GENERAL ACUTE CARE,201 N PARK AVE,NaN,APOPKA,FL,32703,ORANGE,28.680892,-81.508076,10832703,NaN,NaN,10832703,NaN,NaN,NaN,NaN
352,FLORIDA HOSPITAL AT CONNERTON LONG TERM,LONG TERM CARE,9441 HEALTH CENTER DR,NaN,LAND O LAKES,FL,34637,PASCO,28.295273,-82.489914,10934637,5829.0,102026.0,10934637,AdventHealth Connerton (FKA Florida Hospital a...,"9441 Health Center Dr, Land O Lakes, FL",ADVENTHEALTH CONNERTON,"9441 HEALTH CENTER DRIVE, LAND O LAKES, FL"
424,FLORIDA HOSPITAL CARROLLWOOD,GENERAL ACUTE CARE,7171 N DALE MABRY HWY,NaN,TAMPA,FL,33614,HILLSBOROUGH,28.013238,-82.504007,11133614,752.0,100069.0,11133614,AdventHealth Carrollwood (FKA Florida Hospital...,"7171 N Dale Mabry Hwy, Tampa, FL",ADVENTHEALTH CARROLLWOOD,"7171 NORTH DALE MABRY HIGHWAY, TAMPA, FL"


In [16]:
search_ccm_fl = []
for idx, row in ccm_fl.iterrows():
    search_ccm_fl.append((', ').join(row[['Name', 
                                          'DH name lookup', 
                                          'HCRIS name lookup',
#                                           'County'
                                         ]].values.astype('str')).replace('nan',''))

In [17]:
search_ccm_fl_dict = {el:ccm_fl.iloc[idx]['CCM_ID'] for idx, el in enumerate(search_ccm_fl)}

In [20]:
fl_hosp.columns

Index(['County', 'FileNumber', 'ProviderName', 'Bed Census', 'Available',
       'Available Capacity', 'Total Staffed Bed Capacity'],
      dtype='object')

In [21]:
county = ''
for idx, row in fl_hosp.iterrows():
    search_str = (', ').join(fl_hosp.loc[idx,
                                        ['ProviderName',
                                        ]].values.astype(str))

    
    if row['County'] != county:
        search_ccm_county = []
        for _, r in ccm_fl[ccm_fl['County'] == row['County']].iterrows():
#             print(ccm_fl.loc[idx,['Name', 'Address', 'County','Zipcode']])
            search_ccm_county.append((', ').join(r[['Name', 
                                                  'DH name lookup', 
                                                  'HCRIS name lookup',
        #                                           'County'
                                                 ]].values.astype('str')).replace('nan',''))    
    county = row['County']
    
    result = process.extractOne(search_str, search_ccm_county)

    if result != None:
        
        fl_hosp.loc[idx,'Matched Name'] = result[0]
        fl_hosp.loc[idx,'Matched Score'] = result[1]
        fl_hosp.loc[idx,'CCM_ID'] = search_ccm_fl_dict[result[0]]
        fl_hosp.loc[idx,'CCM County'] = county


In [22]:
fl_hosp.shape

(305, 11)

In [23]:
fl_hosp.head()

,County,FileNumber,ProviderName,Bed Census,Available,Available Capacity,Total Staffed Bed Capacity,Matched Name,Matched Score,CCM_ID,CCM County
1,ALACHUA,100113,UF HEALTH SHANDS HOSPITAL,673,373,35.66%,1046,"SHANDS HOSPITAL, UF Health Shands Hospital (FK...",90.000008,35032610.0,ALACHUA
2,ALACHUA,100204,NORTH FLORIDA REGIONAL MEDICAL CENTER,312,123,28.28%,435,"NORTH FLORIDA REGIONAL MEDICAL CENTER, North F...",90.000008,27832605.0,ALACHUA
3,ALACHUA,110025,UF HEALTH REHAB HOSPITAL,45,5,10.00%,50,"SELECT SPECIALTY HOSPITAL GAINESVILLE, UF Heal...",90.000008,34132608.0,ALACHUA
4,ALACHUA,120011,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,41,40,49.38%,81,"MALCOM RANDALL VA MEDICAL CENTER, North Florid...",85.500008,23132608.0,ALACHUA
5,ALACHUA,23960071,SELECT SPECIALTY HOSPITAL GAINESVILLE,42,6,12.50%,48,"SELECT SPECIALTY HOSPITAL GAINESVILLE, UF Heal...",90.000008,34132608.0,ALACHUA


In [141]:
fl_hosp[(fl_hosp['Matched Score'] > 90)][['ProviderName','Matched Name']]

,ProviderName,Matched Name
1,UF HEALTH SHANDS HOSPITAL,"SHANDS HOSPITAL, UF Health Shands Hospital (FK..."
2,NORTH FLORIDA REGIONAL MEDICAL CENTER,"NORTH FLORIDA REGIONAL MEDICAL CENTER, North F..."
3,UF HEALTH REHAB HOSPITAL,"SELECT SPECIALTY HOSPITAL GAINESVILLE, UF Heal..."
5,SELECT SPECIALTY HOSPITAL GAINESVILLE,"SELECT SPECIALTY HOSPITAL GAINESVILLE, UF Heal..."
6,ED FRASER MEMORIAL HOSPITAL,"ED FRASER MEMORIAL HOSPITAL, Baker County Medi..."
...,...,...
299,HALIFAX PSYCHIATRIC CENTER-NORTH,"HALIFAX PSYCHIATRIC CENTER-NORTH, ,"
300,HALIFAX HEALTH MEDICAL CENTER- PORT ORANGE,"HALIFAX HEALTH MEDICAL CENTER- PORT ORANGE, ,"
301,SELECT SPECIALTY HOSPITAL - DAYTONA BEACH,"SELECT SPECIALTY HOSPITAL - DAYTONA BEACH, Sel..."
303,HEALTHMARK REGIONAL MEDICAL CENTER,"HEALTHMARK REGIONAL MEDICAL CENTER, Healthmark..."


In [26]:
# save to file to do manual check and matching
fl_hosp.to_csv(processed_data_path('fl_facilitymatch_debug.csv'))

In [27]:
facility_matched_dict = {k:int(v) for k,v in fl_hosp[(fl_hosp['Matched Score'] >= 0)][['ProviderName','CCM_ID']].values}

In [29]:
# manual matching override

# list of tuples as (ProviderName, CCM_ID)
manual_matches = [
    ("BETHESDA HOSPITAL EAST", 3333435),
    ("UF HEALTH SHANDS PSYCHIATRIC HOSPITAL", 35832606),
    ("CIRCLES OF CARE, INC", 6532901),
    ("FLORIDA MEDICAL CENTER - A CAMPUS OF NORTH SHORE", 28233313),
    ("PHYSICIANS REGIONAL MEDICAL CENTER - COLLIER", 31534114),
    ("ST JOSEPHS HOSPITAL", 38733607),
    ("TAMPA COMMUNITY HOSPITAL- A CAMPUS OF MEMORIAL HOSPITAL OF TAMPA", 40433615),
    ("H LEE MOFFITT CANCER CENTER & RESEARCH INSTITUTE HOSPITAL", 15033612),
    ("STEWARD SEBASTIAN RIVER MEDICAL CENTER", 33932958),
    ("ENCOMPASS HEALTH REHAB HOSPITAL AN AFFILIATE OF MARTIN HEALTH", 36634997),
    ("UNIVERSITY OF MIAMI HOSPITAL AND CLINICS-UHEALTH TOWER", 41233136),
    ("MERCY HOSPITAL, A CAMPUS OF PLANTATION GENERAL HOSPITAL", 26233133),
    ("VARIETY CHILDREN'S HOSPITAL", 26633155),
    ("UNIVERSITY OF MIAMI HOSPITAL AND CLINICS-BASCOM PALMER EYE INST", 433136),
    ("SELECT SPECIALTY HOSPITAL-MIAMI LAKES", 36133016),
    ("ADVENTHEALTH EAST ORLANDO", 11832822),
    ("ADVENTHEALTH WINTER PARK", 43632792),
    ("ARNOLD PALMER MEDICAL CENTER", 532806),
    ("ORLANDO HEALTH DR P PHILLIPS HOSPITAL", 8632819),
    ("ADVENTHEALTH APOPKA", 10832703),
    ("SELECT SPECIALTY HOSPITAL - ORLANDO (NORTH CAMPUS)", 34232803),
    ("ORLANDO HEALTH SOUTH SEMINOLE HOSPITAL", 37332750),
    ("ADVENTHEALTH ALTAMONTE SPRINGS", 10532701)
]
    
for p, ccm_id in manual_matches:
    facility_matched_dict[p] = ccm_id

facility_to_add = [
    ("PALM POINT BEHAVIORAL HEALTH", "CCM-FL-0001"),
    ("FORT LAUDERDALE BEHAVIORAL HEALTH CENTER", "CCM-FL-0002"),
    ("ASCENSION SACRED HEART PENSACOLA", "CCM-FL-0003"),
    ("UF HEALTH LEESBURG SENIOR BEHAVIORAL HEALTH CENTER", "CCM-FL-0004"),
    ("CLEVELAND CLINIC MARTIN SOUTH HOSPITAL", "CCM-FL-0005"),
    ("HALIFAX HEALTH UF HEALTH MEDICAL CENTER OF DELTONA", "CCM-FL-0006")
]

for p, ccm_id in facility_to_add:
    facility_matched_dict[p] = ccm_id

In [40]:
manual_override_data = pd.read_csv(
    external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'),
    dtype={'HCRIS-Provider Number': object, 
           'DH-OBJECTID': object,
           'CCM_ID': object}
)

In [43]:
manual_override_cols = list(manual_override_data.columns)
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'DH-OBJECTID',
 'HCRIS-Provider Number',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [44]:
ccm_csv_df.columns

Index(['Name', 'Hospital Type', 'Address', 'Address_2', 'City', 'State',
       'Zipcode', 'County', 'Latitude', 'Longitude', 'Staffed All Beds',
       'Staffed ICU Beds', 'Licensed All Beds', 'All Bed Occupancy Rate',
       'ICU Bed Occupancy Rate', 'Staffed All Beds - SOURCE',
       'Staffed ICU Beds - SOURCE', 'Licensed All Beds - SOURCE',
       'All Bed Occupancy Rate - SOURCE', 'ICU Bed Occupancy Rate - SOURCE',
       'CCM_ID', 'DH-ID', 'HCRIS-ID', 'HIFLD-ID'],
      dtype='object')

In [45]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'DH-OBJECTID',
 'HCRIS-Provider Number',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [46]:
col_replace = {
    'DH-OBJECTID': 'DH-ID',
    'HCRIS-Provider Number': 'HCRIS-ID',
    'HIFLD-ID': 'HIFLD-ID'
}

manual_override_cols = [col_replace.get(n, n) for n in manual_override_cols]

In [47]:
override_dict = {
    'Staffed All Beds - SOURCE': "FL AHCA 2020-05-03",
    'Staffed ICU Beds - SOURCE': "FL AHCA 2020-05-03",
    'All Bed Occupancy Rate - SOURCE': "FL AHCA 2020-05-03",
    'ICU Bed Occupancy Rate - SOURCE': "FL AHCA 2020-05-03",
    "Manual Override Reason": "More recent data from 2020-05-03",
    "Manual Override New Data Source": "FL AHCA"
}
list(override_dict.keys())

['Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [48]:
fl_hosp['ProviderName']

1                              UF HEALTH SHANDS HOSPITAL
2                  NORTH FLORIDA REGIONAL MEDICAL CENTER
3                               UF HEALTH REHAB HOSPITAL
4                  UF HEALTH SHANDS PSYCHIATRIC HOSPITAL
5                  SELECT SPECIALTY HOSPITAL GAINESVILLE
                             ...                        
301            SELECT SPECIALTY HOSPITAL - DAYTONA BEACH
302    HALIFAX HEALTH UF HEALTH MEDICAL CENTER OF DEL...
303                   HEALTHMARK REGIONAL MEDICAL CENTER
304                 ASCENSION SACRED HEART EMERALD COAST
305                 NORTHWEST FLORIDA COMMUNITY HOSPITAL
Name: ProviderName, Length: 305, dtype: object

In [49]:
manual_override_cols

['Name',
 'Hospital Type',
 'Address',
 'Address_2',
 'City',
 'State',
 'Zipcode',
 'County',
 'Latitude',
 'Longitude',
 'Staffed All Beds',
 'Staffed ICU Beds',
 'Licensed All Beds',
 'All Bed Occupancy Rate',
 'ICU Bed Occupancy Rate',
 'Staffed All Beds - SOURCE',
 'Staffed ICU Beds - SOURCE',
 'Licensed All Beds - SOURCE',
 'All Bed Occupancy Rate - SOURCE',
 'ICU Bed Occupancy Rate - SOURCE',
 'CCM_ID',
 'DH-ID',
 'HCRIS-ID',
 'HIFLD-ID',
 'Manual Override Reason',
 'Manual Override New Data Source']

In [51]:
l = []
for p in fl_hosp['ProviderName']:
    
    ccm_id = str(facility_matched_dict[p])
    found_ccm_record = ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]
    
    # ignore adding new facilities for now, only update existing matched facilities
    if len(ccm_csv_df[ccm_csv_df['CCM_ID'] == ccm_id]) > 0:
        
        hosp_occupancy = fl_hosp[fl_hosp['ProviderName'] == p]['Bed Census'].values[0]/fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]
        icu_occupancy = fl_icu[fl_icu['ProviderName'] == p]['Adult ICU Census'].values[0]/fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
        
        override_dict['Staffed All Beds'] = fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]
        override_dict['Staffed ICU Beds'] = fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
        override_dict['All Bed Occupancy Rate'] = hosp_occupancy
        override_dict['ICU Bed Occupancy Rate'] = icu_occupancy
    
        row_dict = {}

        for col in list(manual_override_cols):
#             print(col)
            # update data for col fields from FL
            if col in list(override_dict.keys()):
                row_dict[col] = override_dict[col]
            # add the other col fields unchanged
            elif len(found_ccm_record[col].values)>0:
                row_dict[col] = found_ccm_record[col].values[0]
            else:
                row_dict[col] = np.nan
        l.append(row_dict)

<ipython-input-51-84d37a8a5f09>:11: RuntimeWarning: invalid value encountered in long_scalars
  icu_occupancy = fl_icu[fl_icu['ProviderName'] == p]['Adult ICU Census'].values[0]/fl_icu[fl_icu['ProviderName'] == p]['Total AdultICU Capacity'].values[0]
<ipython-input-51-84d37a8a5f09>:10: RuntimeWarning: invalid value encountered in long_scalars
  hosp_occupancy = fl_hosp[fl_hosp['ProviderName'] == p]['Bed Census'].values[0]/fl_hosp[fl_hosp['ProviderName'] == p]['Total Staffed Bed Capacity'].values[0]


In [52]:
manual_override_data.shape

(61, 26)

In [53]:
manual_override_data = manual_override_data.append(l)

In [54]:
fl_hosp.columns, fl_icu.columns

(Index(['County', 'FileNumber', 'ProviderName', 'Bed Census', 'Available',
        'Available Capacity', 'Total Staffed Bed Capacity', 'Matched Name',
        'Matched Score', 'CCM_ID', 'CCM County'],
       dtype='object'),
 Index(['County', 'FileNumber', 'ProviderName', 'Adult ICU Census',
        'Available Adult ICU', 'Available Adult ICU%',
        'Total AdultICU Capacity', 'Pediatric ICU Census',
        'Available Pediatric ICU', 'Available Pediatric ICU%',
        'Total PediatricICU Capacity'],
       dtype='object'))

In [55]:
manual_override_data.iloc[-1]

Name                               NORTHWEST FLORIDA COMMUNITY HOSPITAL
Hospital Type                                        GENERAL ACUTE CARE
Address                                               1360 BRICKYARD RD
Address_2                                                           NaN
City                                                            CHIPLEY
State                                                                FL
Zipcode                                                           32428
County                                                       WASHINGTON
Latitude                                                        30.7655
Longitude                                                      -85.5429
Staffed All Beds                                                     59
Staffed ICU Beds                                                      0
Licensed All Beds                                                    59
All Bed Occupancy Rate                                         0

In [56]:
manual_override_data['DH-OBJECTID'] = manual_override_data['DH-OBJECTID'].combine_first(manual_override_data['DH-ID'])
manual_override_data['HCRIS-Provider Number'] = manual_override_data['HCRIS-Provider Number'].combine_first(manual_override_data['HCRIS-ID'])

In [57]:
manual_override_data.drop(['DH-ID', 'HCRIS-ID'], inplace=True, axis=1)

In [59]:
manual_override_data.to_csv(external_data_path('covidcaremap-ushcsc-facility-manual-override.csv'), index=False)